In [44]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

درخواست ۹

به دنبال این هستیم که بدانیم مهم‌ترین عامل خرید لپتاپ چیست؟ این عامل می‌تواند در بین ویژگی‌های لپتاپ مثل ظرفیت حافظه، برند و امثالهم باشد که برای چک کردن آن تعداد فروش را روی نوع مدل لپتاپ جمع می‌کنیم و همبستگی را بررسی می‌کنیم و سپس رگرسیون تعداد فروش مدل روی ویژگی‌هایش می‌بینیم.
از سوی دیگر عامل اثرگذار می‌تواند از جنس زمان خرید مثل ماه خرید باشد که نیاز است روی داده کلی چک شود

In [45]:
all_data = pd.read_csv('sales_data.csv')

In [46]:
all_data.columns

Index(['Order_ID', 'Branch', 'Order_Date', 'Order_Priority', 'Manufacturer',
       'Model_Name', 'Category', 'Screen_Size', 'Screen', 'CPU', 'RAM',
       'Storage', 'GPU', 'OS', 'OS_Version', 'Weight', 'Price', 'Quantity',
       'Discount', 'Total_Price', 'Profit', 'Ship_Duration'],
      dtype='object')

In [47]:
all_data[['year', 'month', 'day']] = all_data.Order_Date.str.split("-",expand=True)
all_data['year']=all_data['year'].astype('int')
all_data['month']=all_data['month'].astype('int')
all_data['day']=all_data['day'].astype('int')


all_data = all_data.loc[((all_data['month']<7) & (all_data['year']==1402)) | (all_data['year']<1402)]
all_data = all_data.loc[((all_data['month']>10) & (all_data['year']==1390)) | (all_data['year']>1390)]

In [48]:
model_data = all_data.groupby(['Model_Name']).sum()[['Quantity','Total_Price']].reset_index()
model_info = all_data.drop_duplicates(subset=['Model_Name'],keep='last')
model_info.drop(columns=['Order_ID','Branch','Order_Date','Order_Priority','Price','Quantity','Total_Price','Ship_Duration'],inplace=True)
model_data = pd.merge(model_info,model_data,how='inner', on='Model_Name')


/var/folders/0r/6nwh7td129301__r2brydjn00000gn/T/ipykernel_95195/4245410701.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_info.drop(columns=['Order_ID','Branch','Order_Date','Order_Priority','Price','Quantity','Total_Price','Ship_Duration'],inplace=True)


In [49]:
model_data[['Screen_Size','maz1']] = model_data.Screen_Size.str.split('"',expand=True)
model_data['Screen_Size']=model_data['Screen_Size'].astype('float')

model_data[['RAM','maz2']] = model_data.RAM.str.split("G",expand=True)
model_data['RAM'] = model_data['RAM'].astype('float')
model_data[['laptop_weight','maz3']] = model_data.Weight.str.split("k",expand=True)
model_data['laptop_weight'] = model_data['laptop_weight'].astype('float')

model_data['CPU_manufacturer'] = model_data.CPU.str.split(" ",expand=True)[0]
#model_data['CPU_manufacturer'].replace('Samsung','other',inplace=True)
#model_data['CPU_manufacturer'].replace('Intel','INTEL',inplace=True)
model_data['graphic_manufacturer'] = model_data.GPU.str.split(" ",expand=True)[0]
#model_data['graphic_manufacturer'].replace('ARM','Other',inplace=True)


In [50]:
model_data.columns
model_data['Price'] = model_data['Total_Price']/model_data['Quantity']
model_data.drop(columns=['Model_Name','Screen','CPU','Storage','GPU','OS_Version','Profit','Weight', 'year', 'month', 'day', 'Total_Price','maz1', 'maz2','maz3' ],inplace=True)
model_data = pd.get_dummies(model_data,columns=['Manufacturer','Category','graphic_manufacturer','OS','CPU_manufacturer', 'graphic_manufacturer'],drop_first=True)


In [51]:
model_data

,Screen_Size,RAM,Discount,Quantity,laptop_weight,Price,Manufacturer_Apple,Manufacturer_Asus,Manufacturer_Chuwi,Manufacturer_Dell,...,OS_Linux,OS_Mac OS,OS_No OS,OS_Windows,OS_macOS,CPU_manufacturer_Intel,CPU_manufacturer_Samsung,graphic_manufacturer_ARM,graphic_manufacturer_Intel,graphic_manufacturer_Nvidia
0,15.6,4.0,2,94,2.10,4.676112e+06,False,False,False,False,...,False,False,False,True,False,True,False,False,True,False
1,14.0,2.0,0,99,1.44,3.192571e+06,False,False,False,False,...,False,False,False,True,False,True,False,False,True,False
2,15.6,4.0,0,111,2.10,3.605140e+06,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,17.3,8.0,0,101,3.40,3.859875e+07,False,False,False,False,...,False,False,False,True,False,True,False,False,False,True
4,11.6,2.0,0,127,0.98,2.755740e+06,False,True,False,False,...,False,False,False,True,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,15.6,4.0,0,27876,2.40,1.325866e+07,False,False,False,False,...,False,False,False,True,False,True,False,False,False,True
614,15.6,4.0,0,1186,2.05,8.539445e+06,False,False,False,False,...,False,False,False,True,False,True,False,False,True,False
615,15.6,8.0,0,11907,1.98,1.091745e+07,False,False,False,True,...,True,False,False,False,False,True,False,False,True,False
616,13.3,8.0,0,1525,1.34,1.455943e+07,False,False,False,False,...,False,False,False,True,False,True,False,False,True,False


ماتریس همبستگی را مشاهده می‌کنیم تا ببینیم وضعیت ارتباط بین ستون تقاضا و دیگر ویژگی‌ها چگونه است

In [57]:
corr_mat_quan=pd.DataFrame(model_data.corr()['Quantity'].sort_values())
corr_mat_quan

,Quantity
Manufacturer_Asus,-0.134360
Manufacturer_MSI,-0.092225
Category_Gaming,-0.076336
RAM,-0.068431
Category_Netbook,-0.059752
Manufacturer_Mediacom,-0.051986
Price,-0.046500
OS_No OS,-0.044939
Manufacturer_Vero,-0.044626
Category_Workstation,-0.044172


مشاهده می‌کنیم که برند دل بودن بیشترین همبستگی را با تقاضا دارد یعنی مستقل از برخی ویژگی‌های مهم لپتاپ همچون حافظه و گرافیک و... و حتی قیمت که برای مصرف‌کننده بسیار مهم است، برند دل دارای ارزش و اهمیت در مقدار خرید افراد است. این مساله می‌تواند به دلیل دردسترس‌بودن، تبلیغات مناسب یا رضایت نسبی با توجه به قیمت در بین مصرف‌کنندگان ایرانی باشد. همچنین با یک فاصله معنی‌دار سیستم عامل مخصوص لپتاپ‌های مک برای خریدار دارای اهمیت است. همچنین از سمت همبستگی منفی هم خود برند لپتاپ دارای اهمیت است. این مشاهده نیازمند بررسی‌های بیشتری است ولی به صورت کاملا اولیه و نادقیق می‌تواند موید این باشد که مردم بیشتر به برند و میزان محبوبیت آن توجه دارند تا به مشخصات جزئی‌تر و فنی‌ لپتاپ